## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator
#import nltk

## Crawler

In [13]:
Dominio = ['http://www.mtgmintcard.com/',
           'https://www.capefeargames.com/',
           'https://www.wizardscupboard.com/',
           'https://www.cardkingdom.com/',
           'http://www.starcitygames.com/',
           'https://www.mtgotraders.com/',
           'https://scryfall.com/','https://abugames.com/']
Dados = []
TempoDominio = []

In [14]:
# Função para marcar a page como visitada.
def Visitou (url,paginasVisitadas):
    if url not in paginasVisitadas:
        return False
    else:
        return True

In [15]:
#Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.head(url,allow_redirects=True)
        ##print(response)
        if response.status_code == rq.codes.ok:
            return True
        else:
            return False
    except:
        return False

In [16]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [17]:
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [18]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## Padrões

In [38]:
def Words():
    goods = ['magic','single','set','catalog','gathering','card','store','foil','product','buylist']
    
    bads = ['pokemom','yugioh','booster','deck','alternate','sculptures','-c-',
           '-t-','box','bundles','contact','complete-set','jpg']
    
    words = [goods,bads]    
    return words

In [39]:
def Padroes():
    
    regex = r'(.+)(magic_singles-)(.+)(\/)(.+)(\/)(\d+)'
    CapeFearGamesMagicCard = re.compile(regex)
    
    regex2 = r'(.+)(store\/)([A-Z]{3}|[A-Z]{1}[0-9]{2})(.+)'
    MtGoTradersMagicCard = re.compile(regex2)
    
    regex3 = r'(.+)(-p-)(\d+)(\.html)'
    WizardCupBoardMagicCard = re.compile(regex3)
    
    regex4 = '(.)*(magic-the-gathering\/)(singles\/)(product-detail\/)(.+)'
    AbuGamesMagicCard = re.compile(regex4)

    regex5 = r'(.+)(mtg\/)(.+)'
    CardKingdomMagicCard = re.compile(regex5)

    regex6 = r'(.+)(magic_the_gathering\/)(product\/)(\d+)(\/\d+)*'
    StarCityMagicCard = re.compile(regex6)

    regex7 = r'(.+)(card)(.+)(\d+)(.+)'
    ScryfallMagicCard = re.compile(regex7)

    regex8 = r'(.+)(mtg\/)(singles\/)(.+\/)(.+\/)(.+)'
    MtgMIntCard = re.compile(regex8)
    
    padroesLinks = [CapeFearGamesMagicCard,CardKingdomMagicCard,AbuGamesMagicCard,ScryfallMagicCard,
                    StarCityMagicCard,WizardCupBoardMagicCard,MtGoTradersMagicCard,MtgMIntCard]
    return padroesLinks

In [40]:
def Score(url,siteInicial,texto):
    score = 0
    
    regexMoney = re.compile(r'(\$)(\d+)(.)(\d{2})')
    
    # Dentro do Dominio +1
    if siteInicial in url:
        score = score + 1
        
    # url contem alguma palavra da lista de palavraes boas que aparecem no link relevante +2   
    for palavra in Words()[0]:
        #casefold ignorar sensitive-case
        if palavra.casefold() in url.casefold():
            score = score + 2
            
    # url fullmatch com link relevante +50
    for padrao in Padroes():
        if padrao.fullmatch(url):
            score = score + 5
    
    #CArdpage tem texto sobre valor da carta. +100
    if regexMoney.match(texto):
        score = score + 100
    
     # url contem alguma palavra da lista de palavras ruins(pokemom, yugioh ...) zera o score.  
    for palavra in Words()[1]:
        if palavra.casefold() in url.casefold():
            score = 0
            return score
    return score

## * Heurística

In [41]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    score = 0
    paginas = [(url,score)]
    paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        #Ordenar a busca pelo Score.
        paginas.sort(key=operator.itemgetter(1),reverse=True)
        
        primeiraPagina = paginas.pop(0)
        
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        # Verifico se a respota da requisição funciona
        while Visitou(primeiraPagina[0].geturl(),paginasVisitadas) == True or primeiraPagina[0].hostname != url.hostname or primeiraPagina[0].hostname == None or robots.allowed(primeiraPagina[0].geturl(), '*') == False or verificarRequest(primeiraPagina[0].geturl()) == False:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
    
        # Verficar se o site é dinamico.
        if (Dominio[7] in url.geturl()):
            texto = CrawlerDinâmico(primeiraPagina[0].geturl())
        else:
            #request - usando o content type
            codigo_fonte = rq.get(primeiraPagina[0].geturl(),headers={'content-type': 'text/html'})
            # texto do codigo_fonte.
            texto = codigo_fonte.text
        soup = BeautifulSoup(texto,'html.parser')

        # Encontrar todos os links da pagina(url) filtrando pelas tags li ul tr td and div.
        for lista in soup.findAll(re.compile(r'li|ul|td|tr|div')):
            for link in lista.findAll('a',href = True):
                
                href = link.get('href')
                # Correção de href relativos para absolutos.
                href = urllib.parse.urljoin(url.geturl(),href)
                
                #Texto na ancora.
                ancoraTexto = link.text
                
                #Texto no Parent da ancora
                parent_text = re.sub('[\s+]', '', link.parent.text)
                if len(parent_text) > 200:
                    parent_text = ''
                
                #concatenar textos da ancora e da tag pai.
                urltexto = ancoraTexto + parent_text

                paginas.append((urlparse(href),Score(href,url.geturl(),urltexto)))
            
        # Marco como visitada a urlparse    
        paginasVisitadas.append(primeiraPagina[0].geturl())
        
        #esperar 300 milisegundos para visitar o prox link.
        time.sleep(.3)
        
        count = count + 1
        print(count,'-',primeiraPagina[0].geturl(),'Score: ',primeiraPagina[1],' Qtd Paginas=', len(paginas))
        
        # Salvar a page localmente
        with open("Paginas/PaginasHeuristica/"+str(url.hostname)+str(count)+".html", 'w' , encoding='utf-8') as outfile:
            outfile.write(texto)
        
        Tabela(count,primeiraPagina[0].geturl(),url.hostname)

### Download pages do Dominio.

In [42]:
for url in Dominio:
    inicio = time.time()
    CardsCrawler(urlparse(url))
    fim = time.time()
    tempo = fim - inicio
    print('Tempo do Site: ',tempo, 'Site: ', url)
    TempoDominio.append(dict(zip(['Dominio','Tempo'],[url,tempo])))

Dominio: http://www.mtgmintcard.com/
1 - http://www.mtgmintcard.com/ Score:  0  Qtd Paginas= 3894
2 - http://www.mtgmintcard.com/mtg/singles/m20 Score:  15  Qtd Paginas= 7033
3 - http://www.mtgmintcard.com/mtg/singles/m20/eng-reg/chandra-awakened-inferno Score:  20  Qtd Paginas= 10319
4 - http://www.mtgmintcard.com/mtg/singles/m20/chi-foil/chandra-awakened-inferno Score:  22  Qtd Paginas= 13206
5 - http://www.mtgmintcard.com/mtg/singles/m20/jap-foil/chandra-awakened-inferno Score:  22  Qtd Paginas= 16007
6 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/chandra-awakened-inferno Score:  22  Qtd Paginas= 19214
7 - http://www.mtgmintcard.com/mtg/singles/m20/chi-foil/chandra-acolyte-of-flame Score:  22  Qtd Paginas= 22493
8 - http://www.mtgmintcard.com/mtg/singles/m20/chi-foil/drawn-from-dreams Score:  22  Qtd Paginas= 25379
9 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/ajani-strength-of-the-pride Score:  22  Qtd Paginas= 28645
10 - http://www.mtgmintcard.com/mtg/singles/m2

77 - http://www.mtgmintcard.com/mtg/singles/m20/jap-foil/flood-of-tears Score:  22  Qtd Paginas= 246240
78 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/temple-of-mystery Score:  22  Qtd Paginas= 249566
79 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/temple-of-silence Score:  22  Qtd Paginas= 252859
80 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/cavalier-of-gales Score:  22  Qtd Paginas= 256143
81 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/temple-of-malady Score:  22  Qtd Paginas= 259469
82 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/temple-of-triumph Score:  22  Qtd Paginas= 262795
83 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/shifting-ceratops Score:  22  Qtd Paginas= 266080
84 - http://www.mtgmintcard.com/mtg/singles/m20/eng-foil/agent-of-treachery Score:  22  Qtd Paginas= 269364
85 - http://www.mtgmintcard.com/mtg/singles/m20/chi-foil/kykar-winds-fury Score:  22  Qtd Paginas= 272102
86 - http://www.mtgmintcard.com/mtg/singl

153 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/arms-dealer Score:  22  Qtd Paginas= 489006
154 - http://www.mtgmintcard.com/mtg/singles/m10/chi-foil/captain-of-the-watch Score:  22  Qtd Paginas= 492232
155 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/rangers-path Score:  22  Qtd Paginas= 495435
156 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/spelltwine Score:  22  Qtd Paginas= 498639
157 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/boundless-realms Score:  22  Qtd Paginas= 501844
158 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/knight-of-glory Score:  22  Qtd Paginas= 505128
159 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/faiths-reward Score:  22  Qtd Paginas= 508332
160 - http://www.mtgmintcard.com/mtg/singles/m13/eng-foil/gilded-lotus Score:  22  Qtd Paginas= 511595
161 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/reliquary-tower Score:  22  Qtd Paginas= 514871
162 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil

230 - http://www.mtgmintcard.com/mtg/singles/m13/eng-foil/vampire-nighthawk Score:  22  Qtd Paginas= 740303
231 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/wild-guess Score:  22  Qtd Paginas= 743542
232 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/sands-of-delirium Score:  22  Qtd Paginas= 746752
233 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/searing-spear Score:  22  Qtd Paginas= 749956
234 - http://www.mtgmintcard.com/mtg/singles/m19/chi-foil/omniscience Score:  22  Qtd Paginas= 752757
235 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/stormtide-leviathan Score:  22  Qtd Paginas= 756040
236 - http://www.mtgmintcard.com/mtg/singles/m12/chi-foil/garruk-primal-hunter Score:  22  Qtd Paginas= 759246
237 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/odric-master-tactician Score:  22  Qtd Paginas= 762409
238 - http://www.mtgmintcard.com/mtg/singles/m13/chi-foil/reverberate Score:  22  Qtd Paginas= 765653
239 - http://www.mtgmintcard.com/mtg/sing

306 - http://www.mtgmintcard.com/mtg/singles/m13/eng-foil/arms-dealer Score:  22  Qtd Paginas= 983320
307 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/ogre-battledriver Score:  22  Qtd Paginas= 986605
308 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/darksteel-forge Score:  22  Qtd Paginas= 989849
309 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/garruks-horde Score:  22  Qtd Paginas= 993094
310 - http://www.mtgmintcard.com/mtg/singles/m11/eng-foil/goblin-balloon-brigade Score:  22  Qtd Paginas= 996298
311 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/nightmare Score:  22  Qtd Paginas= 999623
312 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/clone Score:  22  Qtd Paginas= 1003108
313 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/manaweft-sliver Score:  22  Qtd Paginas= 1006313
314 - http://www.mtgmintcard.com/mtg/singles/m14/eng-foil/mindsparker Score:  22  Qtd Paginas= 1009518
315 - http://www.mtgmintcard.com/mtg/singles/mh1/jap-foil/fi

...

781 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-64082 Score:  14  Qtd Paginas= 423406
782 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-64083 Score:  14  Qtd Paginas= 423927
783 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-64084 Score:  14  Qtd Paginas= 424448
784 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-64085 Score:  14  Qtd Paginas= 424969
785 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-70129 Score:  14  Qtd Paginas= 425490
786 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/ID%23-84354 Score:  14  Qtd Paginas= 426011
787 - https://abugames.com/magic-the-gathering/singles/product-detail/Taiga-%2528WB%2529/Revised/Italian/I

833 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra-%2528Not-Tournament-Legal%2529/Collectors'-Edition-%252D-International/English/ID%23-85973 Score:  14  Qtd Paginas= 452201
834 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra-%2528Not-Tournament-Legal%2529/Collectors'-Edition-%252D-International/English/ID%23-85974 Score:  14  Qtd Paginas= 452722
835 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra/Revised/English/ Score:  14  Qtd Paginas= 453384
836 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra/Revised/English/ID%23-84346 Score:  14  Qtd Paginas= 453905
837 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra/Revised/English/ID%23-87567 Score:  14  Qtd Paginas= 454426
838 - https://abugames.com/magic-the-gathering/singles/product-detail/Tundra/Revised/English/ID%23-88861 Score:  14  Qtd Paginas= 454947
839 - https://abugames.com/magic-the-gathering/singles/product-

887 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/German/ID%23-64111 Score:  14  Qtd Paginas= 482417
888 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/French/ID%23-71378 Score:  14  Qtd Paginas= 482937
889 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/German/ID%23-84363 Score:  14  Qtd Paginas= 483457
890 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/Italian/ Score:  14  Qtd Paginas= 484118
891 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/Italian/ID%23-87571 Score:  14  Qtd Paginas= 484639
892 - https://abugames.com/magic-the-gathering/singles/product-detail/Volcanic-Island-%2528WB%2529/Revised/Italian/ID%23-87572 Score:  14  Qtd Paginas= 485160
893 - https://abugames.com/magic-the-gathering/singles/produ

942 - https://abugames.com/magic-the-gathering/singles/product-detail/Force-of-Nature/4th-Edition/English/ID%23-sa5751 Score:  14  Qtd Paginas= 515667
943 - https://abugames.com/magic-the-gathering/singles/product-detail/Force-of-Nature/4th-Edition/English/ID%23-sa5752 Score:  14  Qtd Paginas= 516175
944 - https://abugames.com/magic-the-gathering/singles/product-detail/Force-of-Nature/4th-Edition/English/ID%23-sa5755 Score:  14  Qtd Paginas= 516683
945 - https://abugames.com/magic-the-gathering/singles/product-detail/Force-of-Nature/4th-Edition/English/ID%23-sa5756 Score:  14  Qtd Paginas= 517191
946 - https://abugames.com/magic-the-gathering/singles/product-detail/Mahamoti-Djinn/4th-Edition/English/ Score:  14  Qtd Paginas= 517713
947 - https://abugames.com/magic-the-gathering/singles/product-detail/Mahamoti-Djinn/4th-Edition/English/ID%23-sa5033 Score:  14  Qtd Paginas= 518221
948 - https://abugames.com/magic-the-gathering/singles/product-detail/Serra-Angel/4th-Edition/English/ Score

999 - https://abugames.com/magic-the-gathering/singles/product-detail/Mox-Jet/Unlimited/English/ID%23-gp28698649 Score:  14  Qtd Paginas= 545444
1000 - https://abugames.com/magic-the-gathering/singles/product-detail/Mox-Jet/Unlimited/English/ID%23-s50334 Score:  14  Qtd Paginas= 545965
Tempo do Site:  14979.784012317657 Site:  https://abugames.com/


### Dados da Heurística

In [43]:
data = pd.DataFrame(Dados,columns=['Número','Link','Site'])
data.head()

,Número,Link,Site
0,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
1,2,http://www.mtgmintcard.com/mtg/singles/m20,www.mtgmintcard.com
2,3,http://www.mtgmintcard.com/mtg/singles/m20/eng...,www.mtgmintcard.com
3,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
4,2,http://www.mtgmintcard.com/mtg/singles/m20,www.mtgmintcard.com


#### Dados - csv

In [44]:
# Dominio da Pagina , Pagina baixada e Ordem do download da pagina 1,2,3,4,5 ...
data.to_csv('CrawlerHeuristica.csv',index_label=False)
data.head()

,Número,Link,Site
0,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
1,2,http://www.mtgmintcard.com/mtg/singles/m20,www.mtgmintcard.com
2,3,http://www.mtgmintcard.com/mtg/singles/m20/eng...,www.mtgmintcard.com
3,1,http://www.mtgmintcard.com/,www.mtgmintcard.com
4,2,http://www.mtgmintcard.com/mtg/singles/m20,www.mtgmintcard.com


### Dados de Tempo da Heurística

In [45]:
TempoDominio

[{'Dominio': 'http://www.mtgmintcard.com/', 'Tempo': 6269.806117773056},
 {'Dominio': 'https://www.capefeargames.com/', 'Tempo': 7493.774067401886},
 {'Dominio': 'https://www.wizardscupboard.com/', 'Tempo': 3443.0994555950165},
 {'Dominio': 'https://www.cardkingdom.com/', 'Tempo': 2917.1280257701874},
 {'Dominio': 'http://www.starcitygames.com/', 'Tempo': 1504.135553598404},
 {'Dominio': 'https://www.mtgotraders.com/', 'Tempo': 3642.0588471889496},
 {'Dominio': 'https://scryfall.com/', 'Tempo': 1271.6132197380066},
 {'Dominio': 'https://abugames.com/', 'Tempo': 14979.784012317657}]

In [46]:
timeData = pd.DataFrame(TempoDominio,columns=['Dominio','Tempo'])
timeData.head()

,Dominio,Tempo
0,http://www.mtgmintcard.com/,6269.806118
1,https://www.capefeargames.com/,7493.774067
2,https://www.wizardscupboard.com/,3443.099456
3,https://www.cardkingdom.com/,2917.128026
4,http://www.starcitygames.com/,1504.135554


#### Média Tempo-Download

In [47]:
print('Média Tempo-Download em Segundos: ',timeData['Tempo'].mean())
print('Média Tempo-Download em Minutos: ',timeData['Tempo'].mean()/60)
print('Média Tempo-Download em Horas: ',(timeData['Tempo'].mean()/60)/60)

Média Tempo-Download em Segundos:  5190.174912422895
Média Tempo-Download em Minutos:  86.50291520704826
Média Tempo-Download em Horas:  1.4417152534508044


#### DataFrame em Minutos

In [48]:
# Convertendo para minutos
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)
timeData

,Dominio,Tempo
0,http://www.mtgmintcard.com/,104.496769
1,https://www.capefeargames.com/,124.896234
2,https://www.wizardscupboard.com/,57.384991
3,https://www.cardkingdom.com/,48.618800
4,http://www.starcitygames.com/,25.068926
5,https://www.mtgotraders.com/,60.700981
6,https://scryfall.com/,21.193554
7,https://abugames.com/,249.663067


#### DataFrame em Horas

In [49]:
# Convertendo para horas
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)
timeData

,Dominio,Tempo
0,http://www.mtgmintcard.com/,1.741613
1,https://www.capefeargames.com/,2.081604
2,https://www.wizardscupboard.com/,0.956417
3,https://www.cardkingdom.com/,0.810313
4,http://www.starcitygames.com/,0.417815
5,https://www.mtgotraders.com/,1.011683
6,https://scryfall.com/,0.353226
7,https://abugames.com/,4.161051


### Tempo-Download

In [50]:
# Tempo de download dos anteriores - tempo em horas
timeData.to_csv('CrawlerHeuristicaTime.csv',index_label=False)
timeData

,Dominio,Tempo
0,http://www.mtgmintcard.com/,1.741613
1,https://www.capefeargames.com/,2.081604
2,https://www.wizardscupboard.com/,0.956417
3,https://www.cardkingdom.com/,0.810313
4,http://www.starcitygames.com/,0.417815
5,https://www.mtgotraders.com/,1.011683
6,https://scryfall.com/,0.353226
7,https://abugames.com/,4.161051
